Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [34]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  loss_rate = tf.constant(0.001)
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + loss_rate * tf.nn.l2_loss(weights)
    

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.821032
Minibatch accuracy: 10.9%
Validation accuracy: 13.3%
Minibatch loss at step 500: 2.436444
Minibatch accuracy: 79.7%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 1.611768
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 1.012356
Minibatch accuracy: 85.9%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.842018
Minibatch accuracy: 82.0%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 0.722064
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.763272
Minibatch accuracy: 82.0%
Validation accuracy: 81.9%
Test accuracy: 88.8%


In [38]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases = tf.Variable(tf.zeros([1024]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  relus = tf.nn.relu(logits)
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  logits2 = tf.matmul(relus, weights2) + biases2
  loss_rate = tf.constant(0.001)
  reg = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) +  loss_rate * reg 
  
  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2) 
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 688.103271
Minibatch accuracy: 6.2%
Validation accuracy: 33.9%
Minibatch loss at step 500: 196.781555
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 116.394722
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 68.617485
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.237312
Minibatch accuracy: 89.8%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 25.172304
Minibatch accuracy: 85.2%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 15.517326
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [39]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases = tf.Variable(tf.zeros([1024]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  relus = tf.nn.relu(logits)
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  logits2 = tf.matmul(relus, weights2) + biases2
  loss_rate = tf.constant(0.001)
  reg = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) +  loss_rate * reg 
  
  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2) 
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 10) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 651.111877
Minibatch accuracy: 10.9%
Validation accuracy: 28.3%
Minibatch loss at step 500: 190.521515
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 115.543694
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 70.072113
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 42.495766
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 25.772846
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 3000: 15.634096
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Test accuracy: 85.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [53]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases = tf.Variable(tf.zeros([1024]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  keep_prob = tf.placeholder(tf.float32)
  dropout_logits = tf.nn.dropout(logits, keep_prob)
    
  relus = tf.nn.relu(dropout_logits)
  
  
    
    
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  logits2 = tf.matmul(relus, weights2) + biases2
  reg_rate = tf.placeholder(tf.float32)
  reg = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels)) +  reg_rate * reg 
  
  

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), weights2) + biases2) 
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)
    
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0, reg_rate: 0.001}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 726.294067
Minibatch accuracy: 4.7%
Validation accuracy: 33.4%
Minibatch loss at step 500: 198.571594
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 115.213623
Minibatch accuracy: 84.4%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 68.683975
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 41.227810
Minibatch accuracy: 89.1%
Validation accuracy: 85.4%
Minibatch loss at step 2500: 25.104429
Minibatch accuracy: 88.3%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 15.475461
Minibatch accuracy: 85.9%
Validation accuracy: 86.9%
Minibatch loss at step 3500: 9.545540
Minibatch accuracy: 88.3%
Validation accuracy: 87.6%
Minibatch loss at step 4000: 5.964510
Minibatch accuracy: 91.4%
Validation accuracy: 87.7%
Minibatch loss at step 4500: 3.790490
Minibatch accuracy: 88.3%
Validation accuracy: 87.4%
Minibatch loss at step 5000: 2.528194
Minibatch accuracy: 89.8%
Validatio

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:


max_steps = 1000
learning_rate = 0.001
dropout = 0.9
data_dir = '/tmp/data'
summaries_dir = '/tmp/mnist_logs'
batch_size = 128
num_features = image_size * image_size

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var, name):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read, and
    adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights, layer_name + '/weights')
        with tf.name_scope("biases"):
            biases = bias_variable([output_dim])
            variable_summaries(biases, layer_name + '/biases')
        with tf.name_scope('Wx_plus_b'):
            activations = tf.matmul(input_tensor, weights) + biases
            tf.histogram_summary(layer_name + '/activations', activations)
        relu = tf.nn.relu(activations, 'relu')
        tf.histogram_summary(layer_name + '/activations_relu', relu)
    return tf.nn.dropout(relu, keep_prob)

graph = tf.Graph()
with graph.as_default():

    with tf.name_scope("input"):
        x = tf.placeholder(tf.float32, [None, 784], "x-input")
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.image_summary('input', image_shaped_input, 10)
        y_ = tf.placeholder(tf.float32, [None, 10], "y-input")
        keep_prob = tf.placeholder(tf.float32)
        tf.scalar_summary('dropout_keep_probability', keep_prob)
        reg_rate = tf.placeholder(tf.float32)    

    layer1 = nn_layer(x, num_features, 1024, 'layer1')
    layer2 = nn_layer(layer1, 1024, num_labels, 'layer2')
    y = tf.nn.softmax(layer2, 'predictions')


    with tf.name_scope('cross_entropy'):
        diff = y_ * tf.log(y)
        with tf.name_scope('total'):
            cross_entropy = -tf.reduce_sum(diff)
        with tf.name_scope('normalized'):
            normalized_cross_entropy = -tf.reduce_mean(diff)
        tf.scalar_summary('cross entropy', normalized_cross_entropy)

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.scalar_summary('accuracy', accuracy)

with  tf.Session(graph=graph) as my_session:
    
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.merge_all_summaries()
    train_writer = tf.train.SummaryWriter(summaries_dir + '/train', my_session.graph)
    test_writer = tf.train.SummaryWriter(summaries_dir + '/test')


    tf.initialize_all_variables().run()


    for step in range(max_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        if step % 10 == 0:
            feed_dict = {x: valid_dataset, y_: valid_labels, keep_prob: 1, reg_rate: 0.001}
            summary, accuracy = my_session.run([merged, accuracy], feed_dict=feed_dict)
            test_writer.add_summary(summary, step)
            print('Validation accuracy at step %s: %s' % (step, accuracy))
        else:
            feed_dict = {x: batch_data, y_: batch_labels, keep_prob: dropout, reg_rate: 0.001}
            summary, accuracy,_ = my_session.run([merged, accuracy, train_step], feed_dict=feed_dict)
            train_writer.add_summary(summary, step)
            print("a")

        #if (step % 500 == 0):
        #  print("Minibatch loss at step %d: %f" % (step, l))
        #  print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        #  print("Validation accuracy: %.1f%%" % accuracy(
        #    valid_prediction.eval(), valid_labels))
    feed_dict = {x: test_dataset, y_: test_labels, keep_prob: 1, reg_rate: 0.001}
    summary, accuracy = my_session.run([merged, accuracy], feed_dict=feed_dict)
    test_writer.add_summary(summary, i)
    print("Test accuracy: %.1f%%" % accuracy)

Validation accuracy at step 0: 0.0887


TypeError: Fetch argument 0.088699996 of 0.088699996 has invalid type <type 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)